In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os
from sklearn.tree import DecisionTreeRegressor

In [7]:
events_2011 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2011/Events_2011.csv')
events_2010 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2010/Events_2010.csv')
events_2012 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2012/Events_2012.csv')
events_2013 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2013/Events_2013.csv')
events_2014 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2014/Events_2014.csv')
events_2015 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2015/Events_2015.csv')
events_2016 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2016/Events_2016.csv')
events_2017 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2017/Events_2017.csv')
events_2018 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2018/Events_2018.csv')
slots1 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/DataFiles 2/NCAATourneySlots.csv')
seeds = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/DataFiles 2/NCAATourneySeedRoundSlots.csv')
reg_compact = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/DataFiles 2/RegularSeasonCompactResults.csv')
teams2 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/DataFiles 2/Teams.csv')
TourneySeeds_1 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/DataFiles/NCAATourneySeeds.csv')
compact1 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/DataFiles/NCAATourneyCompactResults.csv')
reg_compact_2019 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/Prelim2019_RegularSeasonCompactResults.csv')
events_2019 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/PlayByPlay_2019/Events_2019.csv')


In [13]:
slots1 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/DataFiles 2/NCAATourneySlots.csv')
reg_compact1 = pd.read_csv('/Users/andrewcoyne/Desktop/mens-machine-learning-competition-2019/DataFiles 2/RegularSeasonCompactResults.csv')



In [3]:
defense = ['reb_def','reb_off','steal','block','reb_dead']
offense = ['made2_lay','made2_jump','made3_jump','made1_free','made2_tip','made2_dunk','assist']    
deductions =['miss3_jump','miss2_jump','turnover','foul_pers','miss1_free','miss2_lay','miss2_dunk','miss2_tip','foul_tech']
    
def get_team_scores(year_data):
    team_scores = {}
    for event in range(len(year_data)):
        x = year_data['EventTeamID'][event]
        if x in team_scores.keys():
            if year_data['EventType'][event] in defense:
                team_scores[x] += 1
            if year_data['EventType'][event] in offense:
                team_scores[x] += 1
            
        else:
            team_scores[x] = 0
    
    return team_scores

In [4]:
teamscores2010 = get_team_scores(events_2010)
teamscores2011 = get_team_scores(events_2011)
teamscores2012 = get_team_scores(events_2012)
teamscores2013 = get_team_scores(events_2013)
teamscores2014 = get_team_scores(events_2014)
teamscores2015 = get_team_scores(events_2015)
teamscores2016 = get_team_scores(events_2016)
teamscores2017 = get_team_scores(events_2017)
teamscores2018 = get_team_scores(events_2018)

In [159]:
def getitdone(teamscore,year):    
    teamscores = pd.DataFrame(list(teamscore.items()))
    teamscores.columns = ['TeamID','Score']
    teamscores = teamscores.sort_values('TeamID').reset_index(drop=True)

    aseeds = TourneySeeds_1[TourneySeeds_1['Season'] == year].reset_index(drop=True)
    for row in range(len(aseeds)):
        aseeds['Seed'].loc[row] = aseeds['Seed'].loc[row].replace('W','').replace('Y','').replace('X','').replace('Z','').replace('a','').replace('b','')

    compact = compact1[compact1['Season'] == year].reset_index(drop=True)
    for row in range(len(compact)):
        if compact['DayNum'].loc[row] < 136:
            compact = compact.drop([row],axis=0)
    compact = compact.reset_index(drop=True)

    teams = []
    for row in range(len(compact)):
        x = compact['WTeamID'][row]
        y = compact['LTeamID'][row]
        if x not in teams:
            teams.append(x)
        if y not in teams:
            teams.append(y)

    seeds = TourneySeeds_1[TourneySeeds_1['Season'] == year].reset_index(drop=True)
    for row in range(len(seeds)):
        seeds['Seed'].loc[row] = seeds['Seed'].loc[row].replace('a','').replace('b','')
    seeds = seeds[seeds['TeamID'].isin(teams)].reset_index(drop=True)

    teamseeds1 = {}

    for row in range(len(seeds)):
        a = seeds['Seed'].loc[row]
        b = seeds['TeamID'].loc[row]
        teamseeds1[a] = b 

    slots = slots1[slots1['Season'] == year].reset_index(drop=True)
    for row in range(len(slots)):
        if 'a' in slots['StrongSeed'].loc[row]:
            slots = slots.drop(row,axis=0)
    slots = slots.reset_index(drop=True)

    reg_compact = reg_compact1[reg_compact1['Season'] == year].reset_index(drop=True)
    wins = pd.DataFrame(reg_compact['WTeamID'].value_counts()).reset_index()
    wins.columns = ['TeamID','Wins']
    wins = wins.sort_values('TeamID').reset_index(drop=True)
    losses = pd.DataFrame(reg_compact['LTeamID'].value_counts()).reset_index()
    losses.columns = ['TeamID','Wins']
    losses = losses.sort_values('TeamID').reset_index(drop=True)
    record = pd.merge(wins,losses,on='TeamID',how='left').fillna(0)
    record.columns = ['TeamID','Wins','Losses']
    record['% Win'] = record['Wins']/(record['Wins'] + record['Losses'])
    record = record[record['TeamID'].isin(teams)].reset_index(drop=True)

    aseeds = TourneySeeds_1[TourneySeeds_1['Season'] == year].reset_index(drop=True)
    for row in range(len(aseeds)):
        aseeds['Seed'].loc[row] = aseeds['Seed'].loc[row].replace('W','').replace('Y','').replace('X','').replace('Z','').replace('a','').replace('b','')
    aseeds = aseeds[aseeds['TeamID'].isin(teams)].reset_index(drop=True)

    slots['WTeamID'] = slots['StrongSeed']
    slots['StrongName'] = slots['StrongSeed']
    slots['Weakname'] = slots['StrongSeed']
    slots['StrongScore'] = slots['StrongSeed']
    slots['WeakScore'] = slots['WeakSeed']
    slots['HighSeed'] = slots['WeakSeed']
    slots['LowSeed'] = slots['WeakSeed']
    slots['High %'] = slots['WeakSeed']
    slots['Low %'] = slots['WeakSeed']
    slots['Outcome'] = slots['StrongSeed']

    Round1 = slots.loc[0:31]
    Round2 = slots.loc[32:47]
    Round3 = slots.loc[48:55]
    Round4 = slots.loc[56:59]
    Round5 = slots.loc[60:61]

    Round1 = Round1.replace(teamseeds1)

    for each in range(0,32):
        a = teamscores[teamscores['TeamID'] == Round1['StrongSeed'].loc[each]].iloc[0]['Score']
        b = teamscores[teamscores['TeamID'] == Round1['WeakSeed'].loc[each]].iloc[0]['Score']
        c = aseeds[aseeds['TeamID'] == Round1['StrongSeed'].loc[each]].iloc[0]['Seed']
        d = aseeds[aseeds['TeamID'] == Round1['WeakSeed'].loc[each]].iloc[0]['Seed']
        e = record[record['TeamID'] == Round1['StrongSeed'].loc[each]].iloc[0]['% Win']
        f = record[record['TeamID'] == Round1['WeakSeed'].loc[each]].iloc[0]['% Win']
        g = teams2[teams2['TeamID'] == Round1['StrongSeed'].loc[each]].iloc[0]['TeamName']
        h = teams2[teams2['TeamID'] == Round1['WeakSeed'].loc[each]].iloc[0]['TeamName']
        i = compact['LTeamID'].loc[0:31].tolist()
        if Round1['StrongSeed'].loc[each] in i:
            Round1['WTeamID'].loc[each] = Round1['WeakSeed'].loc[each]
        else:
            Round1['WTeamID'].loc[each] = Round1['StrongSeed'].loc[each]
           
        Round1['Outcome'].loc[each] = Round1['WTeamID'].loc[each] 
        
        
        Round1['StrongScore'].loc[each] = a
        Round1['WeakScore'].loc[each] = b
        Round1['HighSeed'].loc[each] = c
        Round1['LowSeed'].loc[each] = d
        Round1['High %'].loc[each] = e
        Round1['Low %'].loc[each] = f
        Round1['StrongName'].loc[each] = g
        Round1['Weakname'].loc[each] = h

    teamseeds2 = {}
    for row in range(len(Round1)):
        a = Round1['Slot'].loc[row]
        b = Round1['WTeamID'].loc[row]
        teamseeds2[a] = b
        
    Round2 = Round2.replace(teamseeds2)    
     
    for each in range(32,48):
        a = teamscores[teamscores['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['Score']
        b = teamscores[teamscores['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['Score']
        c = aseeds[aseeds['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['Seed']
        d = aseeds[aseeds['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['Seed']
        e = record[record['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['% Win']
        f = record[record['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['% Win']
        g = teams2[teams2['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['TeamName']
        h = teams2[teams2['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['TeamName']
        i = compact['LTeamID'].loc[32:47].tolist()
        if Round2['StrongSeed'].loc[each] in i:
            Round2['WTeamID'].loc[each] = Round2['WeakSeed'].loc[each]
        else:
            Round2['WTeamID'].loc[each] = Round2['StrongSeed'].loc[each]
           
        Round2['Outcome'].loc[each] = Round2['WTeamID'].loc[each] 
        
        
        Round2['StrongScore'].loc[each] = a
        Round2['WeakScore'].loc[each] = b
        Round2['HighSeed'].loc[each] = c
        Round2['LowSeed'].loc[each] = d
        Round2['High %'].loc[each] = e
        Round2['Low %'].loc[each] = f
        Round2['StrongName'].loc[each] = g
        Round2['Weakname'].loc[each] = h
        
    teamseeds3 = {}
    for row in range(32,48):
        a = Round2['Slot'].loc[row]
        b = Round2['WTeamID'].loc[row]
        teamseeds3[a] = b
        
    Round3 = Round3.replace(teamseeds3)
    
    for each in range(48,56):
        a = teamscores[teamscores['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['Score']
        b = teamscores[teamscores['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['Score']
        c = aseeds[aseeds['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['Seed']
        d = aseeds[aseeds['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['Seed']
        e = record[record['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['% Win']
        f = record[record['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['% Win']
        g = teams2[teams2['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['TeamName']
        h = teams2[teams2['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['TeamName']
        i = compact['LTeamID'].loc[48:55].tolist()
        if Round3['StrongSeed'].loc[each] in i:
            Round3['WTeamID'].loc[each] = Round3['WeakSeed'].loc[each]
        else:
            Round3['WTeamID'].loc[each] = Round3['StrongSeed'].loc[each]
           
        Round3['Outcome'].loc[each] = Round3['WTeamID'].loc[each] 
        
        
        Round3['StrongScore'].loc[each] = a
        Round3['WeakScore'].loc[each] = b
        Round3['HighSeed'].loc[each] = c
        Round3['LowSeed'].loc[each] = d
        Round3['High %'].loc[each] = e
        Round3['Low %'].loc[each] = f
        Round3['StrongName'].loc[each] = g
        Round3['Weakname'].loc[each] = h
        
    teamseeds4 = {}
    for row in range(48,56):
        a = Round3['Slot'].loc[row]
        b = Round3['WTeamID'].loc[row]
        teamseeds4[a] = b
        
    Round4 = Round4.replace(teamseeds4)
    
    for each in range(56,60):
        a = teamscores[teamscores['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['Score']
        b = teamscores[teamscores['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['Score']
        c = aseeds[aseeds['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['Seed']
        d = aseeds[aseeds['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['Seed']
        e = record[record['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['% Win']
        f = record[record['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['% Win']
        g = teams2[teams2['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['TeamName']
        h = teams2[teams2['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['TeamName']
        i = compact['LTeamID'].loc[56:59].tolist()
        if Round4['StrongSeed'].loc[each] in i:
            Round4['WTeamID'].loc[each] = Round4['WeakSeed'].loc[each]
        else:
            Round4['WTeamID'].loc[each] = Round4['StrongSeed'].loc[each]
           
        Round4['Outcome'].loc[each] = Round4['WTeamID'].loc[each] 
        
        
        Round4['StrongScore'].loc[each] = a
        Round4['WeakScore'].loc[each] = b
        Round4['HighSeed'].loc[each] = c
        Round4['LowSeed'].loc[each] = d
        Round4['High %'].loc[each] = e
        Round4['Low %'].loc[each] = f
        Round4['StrongName'].loc[each] = g
        Round4['Weakname'].loc[each] = h
        
    teamseeds5 = {}
    for row in range(56,60):
        a = Round4['Slot'].loc[row]
        b = Round4['WTeamID'].loc[row]
        teamseeds5[a] = b
        
    Round5 = Round5.replace(teamseeds5)
    
    for each in range(60,62):
        a = teamscores[teamscores['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['Score']
        b = teamscores[teamscores['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['Score']
        c = aseeds[aseeds['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['Seed']
        d = aseeds[aseeds['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['Seed']
        e = record[record['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['% Win']
        f = record[record['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['% Win']
        g = teams2[teams2['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['TeamName']
        h = teams2[teams2['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['TeamName']
        i = compact['LTeamID'].loc[60:61].tolist()
        if Round5['StrongSeed'].loc[each] in i:
            Round5['WTeamID'].loc[each] = Round5['WeakSeed'].loc[each]
        else:
            Round5['WTeamID'].loc[each] = Round5['StrongSeed'].loc[each]
           
        Round5['Outcome'].loc[each] = Round5['WTeamID'].loc[each] 
        
        
        Round5['StrongScore'].loc[each] = a
        Round5['WeakScore'].loc[each] = b
        Round5['HighSeed'].loc[each] = c
        Round5['LowSeed'].loc[each] = d
        Round5['High %'].loc[each] = e
        Round5['Low %'].loc[each] = f
        Round5['StrongName'].loc[each] = g
        Round5['Weakname'].loc[each] = h
        
    end_result = Round1.append([Round2,Round3,Round4,Round5])
        
    return end_result

In [145]:
results2015 = getitdone(teamscores2015,2015)

In [126]:
compact2016 = compact1[compact1['Season'] == 2016].reset_index(drop=True)

In [148]:
results2018 = getitdone(teamscores2018,2018)

In [130]:
results2016 = getitdone(teamscores2016,2016)

In [136]:
compact2015 = compact1[compact1['Season'] == 2015].reset_index(drop=True)
for row in range(len(compact2015)):
    if compact2015['DayNum'].loc[row] < 136:
        compact2015 = compact2015.drop([row],axis=0)
compact2015 = compact2015.reset_index(drop=True)

In [146]:
a = results2015['WTeamID'].loc[0:31].tolist()
b = compact2015['WTeamID'].loc[0:31].tolist()
for each in a:
    if each not in b:
        print(each)
    else:
        print('It is')

It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is
It is


In [150]:
results2017 = getitdone(teamscores2017,2017)

In [160]:
results2014 = getitdone(teamscores2014,2014)

In [162]:
results2013 = getitdone(teamscores2013,2013)

In [164]:
results2012 = getitdone(teamscores2012,2012)

In [166]:
results2011 = getitdone(teamscores2011,2011)

In [168]:
results2010 = getitdone(teamscores2010,2010)

In [171]:
ultimate = results2010.append([results2011,results2012,results2013,results2014,results2015,results2016,results2017,results2018]).reset_index(drop=True)

In [173]:
ultimate.columns

Index(['Season', 'Slot', 'StrongSeed', 'WeakSeed', 'WTeamID', 'StrongName',
       'Weakname', 'StrongScore', 'WeakScore', 'HighSeed', 'LowSeed', 'High %',
       'Low %', 'Outcome'],
      dtype='object')

In [174]:
ncaa_features = ['StrongSeed', 'WeakSeed', 'StrongScore', 'WeakScore', 'HighSeed', 'LowSeed', 'High %',
       'Low %']
y = ultimate.Outcome
X = ultimate[ncaa_features]
ncaa_model = DecisionTreeRegressor(random_state=1)
ncaa_model.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [175]:
print("Making predictions for the following 15 Games:")
print(X.head(15))
print("The predictions are")
print(ncaa_model.predict(X.head(15)))

Making predictions for the following 15 Games:
    StrongSeed  WeakSeed  StrongScore  WeakScore HighSeed LowSeed    High %  \
0         1246      1190         4337       3260       01      16  0.941176   
1         1452      1288         4005       3798       02      15  0.818182   
2         1307      1285         3831       2838       03      14  0.878788   
3         1458      1459         2972       3325       04      13  0.741935   
4         1396      1165         3403       2980       05      12  0.852941   
5         1266      1449         3368       4103       06      11  0.666667   
6         1155      1281         3458       3803       07      10  0.677419   
7         1400      1448         4109       3379       08      09  0.727273   
8         1181      1115         3331       3496       01      16  0.852941   
9         1437      1352         3636       3536       02      15  0.774194   
10        1124      1358         3750       3122       03      14  0.774194   
11   

In [176]:
from sklearn.metrics import mean_absolute_error

predicted_ncaa_outcomes = ncaa_model.predict(X)
mean_absolute_error(y, predicted_ncaa_outcomes)

0.0

In [177]:
from sklearn.metrics import mean_absolute_error

predicted_ncaa_outcomes = ncaa_model.predict(X)
mean_absolute_error(y, predicted_ncaa_outcomes)

from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

ncaa_model = DecisionTreeRegressor()

ncaa_model.fit(train_X, train_y)

val_predictions = ncaa_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

53.05


In [178]:
full2019 = get_team_scores(events_2019)

In [179]:
team_scores_2019 = pd.DataFrame(list(full2019.items()))
team_scores_2019.columns = ['TeamID','Score']
team_scores_2019 = team_scores_2019.sort_values('TeamID').reset_index(drop=True)

In [261]:
reg_compact_2019 = reg_compact_2019[reg_compact_2019['Season'] == 2019].reset_index(drop=True)

In [207]:
teams2019 = {'Virginia':'W01','Gardner Webb':'W16','Mississippi':'W08','Oklahoma':'W09','Wisconsin':'W05','Oregon':'W12','Kansas St':'W04','UC Irvine':'W13','Villanova':'W06',"St Mary's CA":'W11','Purdue':'W03','Old Dominion':'W14','Cincinnati':'W07','Iowa':'W10','Tennessee':'W02','Colgate':'W15','North Carolina':'X01','Iona':'X16','Utah St':'X08','Washington':'X09','Auburn':'X05','New Mexico St':'X12','Kansas':'X04','Northeastern':'X13','Iowa St':'X06','Ohio St':'X11','Houston':'X03','Georgia St':'X14','Wofford':'X07','Seton Hall':'X10','Kentucky':'X02','Abilene Chr':'X15','Duke':'Y01','N Dakota St':'Y16','VA Commonwealth':'Y08','UCF':'Y09','Mississippi St':'Y05','Liberty':'Y12','Virginia Tech':'Y04','St Louis':'Y13','Maryland':'Y06','Belmont':'Y11','LSU':'Y03','Yale':'Y14','Louisville':'Y07','Minnesota':'Y10','Michigan St':'Y02','Bradley':'Y15','Gonzaga':'Z01','F Dickinson':'Z16','Syracuse':'Z08','Baylor':'Z09','Marquette':'Z05','Murray St':'Z12','Florida St':'Z04','Vermont':'Z13','Buffalo':'Z06','Arizona St':'Z11','Texas Tech':'Z03','N Kentucky':'Z14','Nevada':'Z07','Florida':'Z10','Michigan':'Z02','Montana':'Z15'}
teams_2019 = pd.DataFrame(list(teams2019.items()))
teams_2019.columns = ['TeamName','Seed']

In [262]:
wins2019 = pd.DataFrame(reg_compact_2019['WTeamID'].value_counts()).reset_index()
wins2019.columns = ['TeamID','Wins']
wins2019 = wins2019.sort_values('TeamID').reset_index(drop=True)
losses2019 = pd.DataFrame(reg_compact_2019['LTeamID'].value_counts()).reset_index()
losses2019.columns = ['TeamID','Wins']
losses2019 = losses2019.sort_values('TeamID').reset_index(drop=True)
record2019 = pd.merge(wins2019,losses2019,on='TeamID')
record2019.columns = ['TeamID','Wins','Losses']
record2019['% Win'] = record2019['Wins']/(record2019['Wins'] + record2019['Losses'])


In [182]:
slots = slots1[slots1['Season'] == 2018].reset_index(drop=True)
slots = slots.drop([0,1,2,3],axis=0)
slots = slots.reset_index(drop=True)

In [376]:
Round2 = slots.loc[32:47]
Round3 = slots.loc[48:55]
Round4 = slots.loc[56:59]
Round5 = slots.loc[60:61]
Round6 = slots.loc[62]

In [ ]:
final_2019 = final_2019.sort_values('TeamID').reset_index(drop=True)
final_2019 = pd.merge(final_2019,record2019,on='TeamID')

In [223]:
teamnames = list(teams2019.keys())

In [224]:
teams = teams2[['TeamID','TeamName']]
teams = teams[teams['TeamName'].isin(teamnames)].reset_index(drop=True)
the2019 = pd.merge(teams,team_scores_2019, on='TeamID')
the2019 = pd.merge(the2019,record2019, on='TeamID')

In [226]:
len(the2019)

64

In [227]:
mapping_dict = {}
for row in range(len(the2019)):
    a = the2019['TeamName'].loc[row]
    b = the2019['TeamID'].loc[row]
    mapping_dict[a] = b

In [228]:
teams_2019 = teams_2019.replace(mapping_dict)

In [229]:
teams_2019.columns = ['TeamID','Seed']

In [230]:
teamseed = {}

for row in range(len(teams_2019)):
    a = teams_2019['TeamID'].loc[row]
    b = teams_2019['Seed'].loc[row]
    teamseed[b] = a

In [238]:
slots = slots1[slots1['Season'] == 2018].reset_index(drop=True).drop([0,1,2,3],axis=0).reset_index(drop=True)
slots = slots.replace(teamseed)

In [243]:
aseeds = teams_2019
for row in range(len(aseeds)):
    aseeds['Seed'].loc[row] = aseeds['Seed'].loc[row].replace('W','').replace('Y','').replace('X','').replace('Z','').replace('a','').replace('b','')


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [375]:
slots['StrongName'] = slots['StrongSeed']
slots['Weakname'] = slots['StrongSeed']
slots['StrongScore'] = slots['StrongSeed']
slots['WeakScore'] = slots['WeakSeed']
slots['HighSeed'] = slots['WeakSeed']
slots['LowSeed'] = slots['WeakSeed']
slots['High %'] = slots['WeakSeed']
slots['Low %'] = slots['WeakSeed']


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [251]:
slots = slots[['Season', 'Slot', 'StrongSeed', 'WeakSeed', 'WTeamID', 'StrongName',
       'Weakname', 'StrongScore', 'WeakScore', 'HighSeed', 'LowSeed', 'High %',
       'Low %']]

In [265]:
for each in range(0,32):
    a = team_scores_2019[team_scores_2019['TeamID'] == Round1['StrongSeed'].loc[each]].iloc[0]['Score']
    b = team_scores_2019[team_scores_2019['TeamID'] == Round1['WeakSeed'].loc[each]].iloc[0]['Score']
    c = aseeds[aseeds['TeamID'] == Round1['StrongSeed'].loc[each]].iloc[0]['Seed']
    d = aseeds[aseeds['TeamID'] == Round1['WeakSeed'].loc[each]].iloc[0]['Seed']
    e = record2019[record2019['TeamID'] == Round1['StrongSeed'].loc[each]].iloc[0]['% Win']
    f = record2019[record2019['TeamID'] == Round1['WeakSeed'].loc[each]].iloc[0]['% Win']
    g = teams2[teams2['TeamID'] == Round1['StrongSeed'].loc[each]].iloc[0]['TeamName']
    h = teams2[teams2['TeamID'] == Round1['WeakSeed'].loc[each]].iloc[0]['TeamName']


    Round1['StrongScore'].loc[each] = a
    Round1['WeakScore'].loc[each] = b
    Round1['HighSeed'].loc[each] = c
    Round1['LowSeed'].loc[each] = d
    Round1['High %'].loc[each] = e
    Round1['Low %'].loc[each] = f
    Round1['StrongName'].loc[each] = g
    Round1['Weakname'].loc[each] = h
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %
0,2018,R1W1,1438,1205,1438,Virginia,Gardner Webb,3162,3107,01,16,0.933333,0.645161
1,2018,R1W2,1397,1159,1397,Tennessee,Colgate,3754,3372,02,15,0.866667,0.69697
2,2018,R1W3,1345,1330,1345,Purdue,Old Dominion,3343,3372,03,14,0.741935,0.741935
3,2018,R1W4,1243,1414,1243,Kansas St,UC Irvine,3144,3533,04,13,0.774194,0.83871
4,2018,R1W5,1458,1332,1458,Wisconsin,Oregon,3179,3512,05,12,0.709677,0.625
5,2018,R1W6,1437,1388,1437,Villanova,St Mary's CA,3402,3118,06,11,0.709677,0.666667
6,2018,R1W7,1153,1234,1153,Cincinnati,Iowa,3588,3572,07,10,0.806452,0.677419
7,2018,R1W8,1279,1328,1279,Mississippi,Oklahoma,3291,3225,08,09,0.645161,0.59375
8,2018,R1X1,1314,1233,1314,North Carolina,Iona,4050,3256,01,16,0.83871,0.53125
9,2018,R1X2,1246,1101,1246,Kentucky,Abilene Chr,3656,2858,02,15,0.83871,0.777778


In [285]:
Z = Round1[ncaa_features]
prediction = ncaa_model.predict(Z)

In [272]:
Round1['Outcome'] = prediction

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [361]:

Round1['Outcome'] = prediction
Round1['Next'] = Round1['WeakSeed']



for row in range(len(Round1)):
    a = Round1['StrongSeed'].loc[row]
    b = Round1['WeakSeed'].loc[row]
    c = Round1['Outcome'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round1['Next'].loc[row] = b
    else:
        Round1['Next'].loc[row] = a
        

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [278]:
teamseed2 = {}
for row in range(len(Round1)):
    a = Round1['Slot'].loc[row]
    b = Round1['Next'].loc[row]
    teamseed2[a] = b

In [281]:
Round2 = Round2.replace(teamseed2)

In [282]:
for each in range(32,48):
    a = team_scores_2019[team_scores_2019['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['Score']
    b = team_scores_2019[team_scores_2019['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['Score']
    c = aseeds[aseeds['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['Seed']
    d = aseeds[aseeds['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['Seed']
    e = record2019[record2019['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['% Win']
    f = record2019[record2019['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['% Win']
    g = teams2[teams2['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['TeamName']
    h = teams2[teams2['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['TeamName']


    Round2['StrongScore'].loc[each] = a
    Round2['WeakScore'].loc[each] = b
    Round2['HighSeed'].loc[each] = c
    Round2['LowSeed'].loc[each] = d
    Round2['High %'].loc[each] = e
    Round2['Low %'].loc[each] = f
    Round2['StrongName'].loc[each] = g
    Round2['Weakname'].loc[each] = h

In [286]:
Y = Round2[ncaa_features]
prediction = ncaa_model.predict(Y)

In [288]:
Round2['Outcome'] = prediction2
Round2['Next'] = Round2['WeakSeed']

for row in range(32,48):
    a = Round2['StrongSeed'].loc[row]
    b = Round2['WeakSeed'].loc[row]
    c = Round2['Outcome'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round2['Next'].loc[row] = b
    else:
        Round2['Next'].loc[row] = a

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [289]:
Round2

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,Outcome,Next
32,2018,R2W1,1438,1279,1438,Virginia,Mississippi,3162,3291,01,08,0.933333,0.645161,1437.0,1438
33,2018,R2W2,1159,1153,1159,Colgate,Cincinnati,3372,3588,15,07,0.696970,0.806452,1161.0,1159
34,2018,R2W3,1345,1437,1345,Purdue,Villanova,3343,3402,03,06,0.741935,0.709677,1393.0,1437
35,2018,R2W4,1243,1458,1243,Kansas St,Wisconsin,3144,3179,04,05,0.774194,0.709677,1247.0,1243
36,2018,R2X1,1314,1449,1314,North Carolina,Washington,4050,3369,01,09,0.838710,0.774194,1314.0,1314
37,2018,R2X2,1246,1459,1246,Kentucky,Wofford,3656,3084,02,07,0.838710,0.866667,1455.0,1459
38,2018,R2X3,1222,1235,1222,Houston,Iowa St,3717,3597,03,06,0.935484,0.645161,1222.0,1222
39,2018,R2X4,1242,1120,1242,Kansas,Auburn,3603,3583,04,05,0.741935,0.700000,1242.0,1242
40,2018,R2Y1,1181,1433,1181,Duke,VA Commonwealth,4183,3369,01,08,0.838710,0.806452,1181.0,1181
41,2018,R2Y2,1277,1257,1277,Michigan St,Louisville,3940,3376,02,07,0.806452,0.625000,1274.0,1277


In [291]:
teamseed3 = {}
for row in range(32,48):
    a = Round2['Slot'].loc[row]
    b = Round2['Next'].loc[row]
    teamseed3[a] = b

In [292]:
Round3 = Round3.replace(teamseed3)

In [300]:
for each in range(48,56):
    a = team_scores_2019[team_scores_2019['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['Score']
    b = team_scores_2019[team_scores_2019['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['Score']
    c = aseeds[aseeds['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['Seed']
    d = aseeds[aseeds['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['Seed']
    e = record2019[record2019['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['% Win']
    f = record2019[record2019['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['% Win']
    g = teams2[teams2['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['TeamName']
    h = teams2[teams2['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['TeamName']


    Round3['StrongScore'].loc[each] = a
    Round3['WeakScore'].loc[each] = b
    Round3['HighSeed'].loc[each] = c
    Round3['LowSeed'].loc[each] = d
    Round3['High %'].loc[each] = e
    Round3['Low %'].loc[each] = f
    Round3['StrongName'].loc[each] = g
    Round3['Weakname'].loc[each] = h

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [302]:
X = Round3[ncaa_features]
prediction3 = ncaa_model.predict(X)

Round3['Outcome'] = prediction3
Round3['Next'] = Round3['WeakSeed']

for row in range(48,56):
    a = Round3['StrongSeed'].loc[row]
    b = Round3['WeakSeed'].loc[row]
    c = Round3['Outcome'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round3['Next'].loc[row] = b
    else:
        Round3['Next'].loc[row] = a

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [303]:
Round3

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,Outcome,Next
48,2018,R3W1,1438,1243,1438,Virginia,Kansas St,3162,3144,01,04,0.933333,0.774194,1437.0,1438
49,2018,R3W2,1159,1437,1159,Colgate,Villanova,3372,3402,15,06,0.696970,0.709677,1161.0,1159
50,2018,R3X1,1314,1242,1314,North Carolina,Kansas,4050,3603,01,04,0.838710,0.741935,1246.0,1242
51,2018,R3X2,1459,1222,1459,Wofford,Houston,3084,3717,07,03,0.866667,0.935484,1210.0,1222
52,2018,R3Y1,1181,1387,1181,Duke,St Louis,4183,3691,01,13,0.838710,0.612903,1181.0,1181
53,2018,R3Y2,1277,1261,1277,Michigan St,LSU,3940,3650,02,03,0.806452,0.838710,1277.0,1277
54,2018,R3Z1,1211,1199,1211,Gonzaga,Florida St,3996,3611,01,04,0.909091,0.806452,1211.0,1211
55,2018,R3Z2,1196,1403,1196,Florida,Texas Tech,3246,3289,10,03,0.548387,0.838710,1196.0,1196


In [304]:
teamseed4 = {}
for row in range(48,56):
    a = Round3['Slot'].loc[row]
    b = Round3['Next'].loc[row]
    teamseed4[a] = b
    
Round4 = Round4.replace(teamseed4)

In [306]:
for each in range(56,60):
    a = team_scores_2019[team_scores_2019['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['Score']
    b = team_scores_2019[team_scores_2019['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['Score']
    c = aseeds[aseeds['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['Seed']
    d = aseeds[aseeds['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['Seed']
    e = record2019[record2019['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['% Win']
    f = record2019[record2019['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['% Win']
    g = teams2[teams2['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['TeamName']
    h = teams2[teams2['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['TeamName']


    Round4['StrongScore'].loc[each] = a
    Round4['WeakScore'].loc[each] = b
    Round4['HighSeed'].loc[each] = c
    Round4['LowSeed'].loc[each] = d
    Round4['High %'].loc[each] = e
    Round4['Low %'].loc[each] = f
    Round4['StrongName'].loc[each] = g
    Round4['Weakname'].loc[each] = h
    
W = Round4[ncaa_features]
prediction4 = ncaa_model.predict(W)

Round4['Outcome'] = prediction4
Round4['Next'] = Round4['WeakSeed']

for row in range(56,60):
    a = Round4['StrongSeed'].loc[row]
    b = Round4['WeakSeed'].loc[row]
    c = Round4['Outcome'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round4['Next'].loc[row] = b
    else:
        Round4['Next'].loc[row] = a

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [307]:
Round4

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,Outcome,Next
56,2018,R4W1,1438,1159,1438,Virginia,Colgate,3162,3372,01,15,0.933333,0.696970,1438.0,1438
57,2018,R4X1,1242,1222,1242,Kansas,Houston,3603,3717,04,03,0.741935,0.935484,1257.0,1242
58,2018,R4Y1,1181,1277,1181,Duke,Michigan St,4183,3940,01,02,0.838710,0.806452,1242.0,1277
59,2018,R4Z1,1211,1196,1211,Gonzaga,Florida,3996,3246,01,10,0.909091,0.548387,1211.0,1211


In [309]:
teamseed5 = {}
for row in range(56,60):
    a = Round4['Slot'].loc[row]
    b = Round4['Next'].loc[row]
    teamseed5[a] = b
    
Round5 = Round5.replace(teamseed5)

In [312]:
for each in range(60,62):
    a = team_scores_2019[team_scores_2019['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['Score']
    b = team_scores_2019[team_scores_2019['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['Score']
    c = aseeds[aseeds['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['Seed']
    d = aseeds[aseeds['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['Seed']
    e = record2019[record2019['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['% Win']
    f = record2019[record2019['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['% Win']
    g = teams2[teams2['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['TeamName']
    h = teams2[teams2['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['TeamName']


    Round5['StrongScore'].loc[each] = a
    Round5['WeakScore'].loc[each] = b
    Round5['HighSeed'].loc[each] = c
    Round5['LowSeed'].loc[each] = d
    Round5['High %'].loc[each] = e
    Round5['Low %'].loc[each] = f
    Round5['StrongName'].loc[each] = g
    Round5['Weakname'].loc[each] = h
    
V = Round5[ncaa_features]
prediction5 = ncaa_model.predict(V)

Round5['Outcome'] = prediction5
Round5['Next'] = Round5['WeakSeed']

for row in range(60,62):
    a = Round5['StrongSeed'].loc[row]
    b = Round5['WeakSeed'].loc[row]
    c = Round5['Outcome'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round5['Next'].loc[row] = b
    else:
        Round5['Next'].loc[row] = a

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [313]:
Round5

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,Outcome,Next
60,2018,R5WX,1438,1242,1438,Virginia,Kansas,3162,3603,01,04,0.933333,0.741935,1438.0,1438
61,2018,R5YZ,1277,1211,1277,Michigan St,Gonzaga,3940,3996,02,01,0.806452,0.909091,1139.0,1211


In [331]:
type(Round1['HighSeed'].loc[5])

str

In [314]:
final = slots.loc[62]

In [316]:
final = slots.loc[62]
teamseed6 = {}
for row in range(60,62):
    a = Round5['Slot'].loc[row]
    b = Round5['Next'].loc[row]
    teamseed6[a] = b
    
final = final.replace(teamseed6)

final['StrongScore'] = 3162
final['WeakScore'] = 3996
final['HighSeed'] = '01'
final['LowSeed'] = '011
final['High %'] = 0.933333
final['Low %'] = 0.909091
final['StrongName'] = 'Virginia'
final['Weakname'] = 'Gonzaga'

In [327]:
final['StrongScore'] = 3162
final['WeakScore'] = 3996
final['HighSeed'] = '01'
final['LowSeed'] = '011
final['High %'] = 0.933333
final['Low %'] = 0.909091
final['StrongName'] = 'Virginia'
final['Weakname'] = 'Gonzaga'

In [332]:
U = final[ncaa_features]
final_prediction = ncaa_model.predict(U)

ValueError: Expected 2D array, got 1D array instead:
array=[1.43800e+03 1.21100e+03 3.16200e+03 3.99600e+03 1.00000e+00 1.00000e+00
 9.33333e-01 9.09091e-01].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [335]:
U = pd.DataFrame(U)

In [342]:
U = pd.DataFrame(U)
U = U.transpose()
final_prediction = ncaa_model.predict(U)

In [344]:
final_prediction = ncaa_model.predict(U)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

In [345]:
final_prediction

array([1211.])

In [347]:
U

,StrongSeed,WeakSeed,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %
62,1438,1211,3162,3996,1,1,0.933333,0.909091


In [ ]:
ncaa_features = ['StrongSeed', 'WeakSeed', 'StrongScore', 'WeakScore', 'HighSeed', 'LowSeed', 'High %',
       'Low %']
y = ultimate.Outcome
X = ultimate[ncaa_features]
ncaa_model = DecisionTreeRegressor(random_state=1)
ncaa_model.fit(X,y)

print("Making predictions for the following 15 Games:")
print(X.head(15))
print("The predictions are")
print(ncaa_model.predict(X.head(15)))

from sklearn.metrics import mean_absolute_error

predicted_ncaa_outcomes = ncaa_model.predict(X)
mean_absolute_error(y, predicted_ncaa_outcomes)

from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

ncaa_model = DecisionTreeRegressor()

ncaa_model.fit(train_X, train_y)

val_predictions = ncaa_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))




In [348]:
val_mae = mean_absolute_error(val_predictions, val_y)

In [352]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  59
Max leaf nodes: 50  		 Mean Absolute Error:  53
Max leaf nodes: 500  		 Mean Absolute Error:  52
Max leaf nodes: 5000  		 Mean Absolute Error:  52


In [353]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
ncaa_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, ncaa_preds))


48.5492857142857


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [357]:
Round1['OutcomeForest'] = forest_model.predict(Z)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [362]:

for row in range(len(Round1)):
    a = Round1['StrongSeed'].loc[row]
    b = Round1['WeakSeed'].loc[row]
    c = Round1['OutcomeForest'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round1['NextForest'].loc[row] = b
    else:
        Round1['NextForest'].loc[row] = a

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [363]:
Round1

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,Outcome,Next,NextForest,OutcomeForest
0,2018,R1W1,1438,1205,1438,Virginia,Gardner Webb,3162,3107,01,16,0.933333,0.645161,1428.0,1438,1438.0,1361.6
1,2018,R1W2,1397,1159,1397,Tennessee,Colgate,3754,3372,02,15,0.866667,0.69697,1112.0,1159,1397.0,1334.0
2,2018,R1W3,1345,1330,1345,Purdue,Old Dominion,3343,3372,03,14,0.741935,0.741935,1361.0,1345,1330.0,1334.8
3,2018,R1W4,1243,1414,1243,Kansas St,UC Irvine,3144,3533,04,13,0.774194,0.83871,1243.0,1243,1243.0,1285.0
4,2018,R1W5,1458,1332,1458,Wisconsin,Oregon,3179,3512,05,12,0.709677,0.625,1458.0,1458,1458.0,1415.4
5,2018,R1W6,1437,1388,1437,Villanova,St Mary's CA,3402,3118,06,11,0.709677,0.666667,1437.0,1437,1388.0,1393.9
6,2018,R1W7,1153,1234,1153,Cincinnati,Iowa,3588,3572,07,10,0.806452,0.677419,1153.0,1153,1153.0,1156.6
7,2018,R1W8,1279,1328,1279,Mississippi,Oklahoma,3291,3225,08,09,0.645161,0.59375,1292.0,1279,1279.0,1286.9
8,2018,R1X1,1314,1233,1314,North Carolina,Iona,4050,3256,01,16,0.83871,0.53125,1314.0,1314,1314.0,1321.0
9,2018,R1X2,1246,1101,1246,Kentucky,Abilene Chr,3656,2858,02,15,0.83871,0.777778,1246.0,1246,1246.0,1231.9


In [364]:
Round2

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,Outcome,Next
32,2018,R2W1,1438,1279,1438,Virginia,Mississippi,3162,3291,01,08,0.933333,0.645161,1437.0,1438
33,2018,R2W2,1159,1153,1159,Colgate,Cincinnati,3372,3588,15,07,0.696970,0.806452,1161.0,1159
34,2018,R2W3,1345,1437,1345,Purdue,Villanova,3343,3402,03,06,0.741935,0.709677,1393.0,1437
35,2018,R2W4,1243,1458,1243,Kansas St,Wisconsin,3144,3179,04,05,0.774194,0.709677,1247.0,1243
36,2018,R2X1,1314,1449,1314,North Carolina,Washington,4050,3369,01,09,0.838710,0.774194,1314.0,1314
37,2018,R2X2,1246,1459,1246,Kentucky,Wofford,3656,3084,02,07,0.838710,0.866667,1455.0,1459
38,2018,R2X3,1222,1235,1222,Houston,Iowa St,3717,3597,03,06,0.935484,0.645161,1222.0,1222
39,2018,R2X4,1242,1120,1242,Kansas,Auburn,3603,3583,04,05,0.741935,0.700000,1242.0,1242
40,2018,R2Y1,1181,1433,1181,Duke,VA Commonwealth,4183,3369,01,08,0.838710,0.806452,1181.0,1181
41,2018,R2Y2,1277,1257,1277,Michigan St,Louisville,3940,3376,02,07,0.806452,0.625000,1274.0,1277


In [377]:
teamseed2 = {}
for row in range(len(Round1)):
    a = Round1['Slot'].loc[row]
    b = Round1['NextForest'].loc[row]
    teamseed2[a] = b
    
Round2 = Round2.replace(teamseed2)

for each in range(32,48):
    a = team_scores_2019[team_scores_2019['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['Score']
    b = team_scores_2019[team_scores_2019['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['Score']
    c = aseeds[aseeds['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['Seed']
    d = aseeds[aseeds['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['Seed']
    e = record2019[record2019['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['% Win']
    f = record2019[record2019['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['% Win']
    g = teams2[teams2['TeamID'] == Round2['StrongSeed'].loc[each]].iloc[0]['TeamName']
    h = teams2[teams2['TeamID'] == Round2['WeakSeed'].loc[each]].iloc[0]['TeamName']


    Round2['StrongScore'].loc[each] = a
    Round2['WeakScore'].loc[each] = b
    Round2['HighSeed'].loc[each] = c
    Round2['LowSeed'].loc[each] = d
    Round2['High %'].loc[each] = e
    Round2['Low %'].loc[each] = f
    Round2['StrongName'].loc[each] = g
    Round2['Weakname'].loc[each] = h

Y = Round2[ncaa_features]

Round2['NextForest'] = Round2['WeakSeed']
Round2['OutcomeForest'] = ncaa_model.predict(Y)

for row in range(32,48):
    a = Round2['StrongSeed'].loc[row]
    b = Round2['WeakSeed'].loc[row]
    c = Round2['OutcomeForest'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round2['NextForest'].loc[row] = b
    else:
        Round2['NextForest'].loc[row] = a

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [378]:
Round2

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,NextForest,OutcomeForest
32,2018,R2W1,1438.0,1279.0,1438.0,Virginia,Mississippi,3162.0,3291.0,01,08,0.933333,0.645161,1438.0,1437.0
33,2018,R2W2,1397.0,1153.0,1397.0,Tennessee,Cincinnati,3754.0,3588.0,02,07,0.866667,0.806452,1153.0,1112.0
34,2018,R2W3,1330.0,1388.0,1330.0,Old Dominion,St Mary's CA,3372.0,3118.0,14,11,0.741935,0.666667,1330.0,1332.0
35,2018,R2W4,1243.0,1458.0,1243.0,Kansas St,Wisconsin,3144.0,3179.0,04,05,0.774194,0.709677,1243.0,1247.0
36,2018,R2X1,1314.0,1429.0,1314.0,North Carolina,Utah St,4050.0,3752.0,01,08,0.838710,0.800000,1314.0,1314.0
37,2018,R2X2,1246.0,1459.0,1246.0,Kentucky,Wofford,3656.0,3084.0,02,07,0.838710,0.866667,1459.0,1455.0
38,2018,R2X3,1222.0,1235.0,1222.0,Houston,Iowa St,3717.0,3597.0,03,06,0.935484,0.645161,1222.0,1222.0
39,2018,R2X4,1242.0,1120.0,1242.0,Kansas,Auburn,3603.0,3583.0,04,05,0.741935,0.700000,1242.0,1242.0
40,2018,R2Y1,1181.0,1416.0,1181.0,Duke,UCF,4183.0,3258.0,01,09,0.838710,0.766667,1181.0,1181.0
41,2018,R2Y2,1277.0,1278.0,1277.0,Michigan St,Minnesota,3940.0,3571.0,02,10,0.806452,0.612903,1277.0,1274.0


In [379]:
teamseed3 = {}
for row in range(32,48):
    a = Round2['Slot'].loc[row]
    b = Round2['NextForest'].loc[row]
    teamseed3[a] = b
    
Round3 = Round3.replace(teamseed3)

for each in range(48,56):
    a = team_scores_2019[team_scores_2019['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['Score']
    b = team_scores_2019[team_scores_2019['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['Score']
    c = aseeds[aseeds['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['Seed']
    d = aseeds[aseeds['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['Seed']
    e = record2019[record2019['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['% Win']
    f = record2019[record2019['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['% Win']
    g = teams2[teams2['TeamID'] == Round3['StrongSeed'].loc[each]].iloc[0]['TeamName']
    h = teams2[teams2['TeamID'] == Round3['WeakSeed'].loc[each]].iloc[0]['TeamName']


    Round3['StrongScore'].loc[each] = a
    Round3['WeakScore'].loc[each] = b
    Round3['HighSeed'].loc[each] = c
    Round3['LowSeed'].loc[each] = d
    Round3['High %'].loc[each] = e
    Round3['Low %'].loc[each] = f
    Round3['StrongName'].loc[each] = g
    Round3['Weakname'].loc[each] = h

W = Round3[ncaa_features]

Round3['NextForest'] = Round3['WeakSeed']
Round3['OutcomeForest'] = ncaa_model.predict(W)

for row in range(48,56):
    a = Round3['StrongSeed'].loc[row]
    b = Round3['WeakSeed'].loc[row]
    c = Round3['OutcomeForest'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round3['NextForest'].loc[row] = b
    else:
        Round3['NextForest'].loc[row] = a

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [380]:
Round3

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,NextForest,OutcomeForest
48,2018,R3W1,1438.0,1243.0,1438.0,Virginia,Kansas St,3162.0,3144.0,01,04,0.933333,0.774194,1438.0,1437.0
49,2018,R3W2,1153.0,1330.0,1153.0,Cincinnati,Old Dominion,3588.0,3372.0,07,14,0.806452,0.741935,1153.0,1153.0
50,2018,R3X1,1314.0,1242.0,1314.0,North Carolina,Kansas,4050.0,3603.0,01,04,0.838710,0.741935,1242.0,1246.0
51,2018,R3X2,1459.0,1222.0,1459.0,Wofford,Houston,3084.0,3717.0,07,03,0.866667,0.935484,1222.0,1210.0
52,2018,R3Y1,1181.0,1387.0,1181.0,Duke,St Louis,4183.0,3691.0,01,13,0.838710,0.612903,1181.0,1181.0
53,2018,R3Y2,1277.0,1268.0,1277.0,Michigan St,Maryland,3940.0,3288.0,02,06,0.806452,0.709677,1277.0,1274.0
54,2018,R3Z1,1211.0,1293.0,1211.0,Gonzaga,Murray St,3996.0,3286.0,01,12,0.909091,0.862069,1211.0,1211.0
55,2018,R3Z2,1285.0,1403.0,1285.0,Montana,Texas Tech,3214.0,3289.0,15,03,0.724138,0.838710,1285.0,1276.0


In [381]:
teamseed4 = {}
for row in range(48,56):
    a = Round3['Slot'].loc[row]
    b = Round3['NextForest'].loc[row]
    teamseed4[a] = b
    
Round4 = Round4.replace(teamseed4)

for each in range(56,60):
    a = team_scores_2019[team_scores_2019['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['Score']
    b = team_scores_2019[team_scores_2019['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['Score']
    c = aseeds[aseeds['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['Seed']
    d = aseeds[aseeds['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['Seed']
    e = record2019[record2019['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['% Win']
    f = record2019[record2019['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['% Win']
    g = teams2[teams2['TeamID'] == Round4['StrongSeed'].loc[each]].iloc[0]['TeamName']
    h = teams2[teams2['TeamID'] == Round4['WeakSeed'].loc[each]].iloc[0]['TeamName']


    Round4['StrongScore'].loc[each] = a
    Round4['WeakScore'].loc[each] = b
    Round4['HighSeed'].loc[each] = c
    Round4['LowSeed'].loc[each] = d
    Round4['High %'].loc[each] = e
    Round4['Low %'].loc[each] = f
    Round4['StrongName'].loc[each] = g
    Round4['Weakname'].loc[each] = h

V = Round4[ncaa_features]

Round4['NextForest'] = Round4['WeakSeed']
Round4['OutcomeForest'] = ncaa_model.predict(V)

for row in range(56,60):
    a = Round4['StrongSeed'].loc[row]
    b = Round4['WeakSeed'].loc[row]
    c = Round4['OutcomeForest'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round4['NextForest'].loc[row] = b
    else:
        Round4['NextForest'].loc[row] = a

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [382]:
Round4

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,NextForest,OutcomeForest
56,2018,R4W1,1438.0,1153.0,1438.0,Virginia,Cincinnati,3162.0,3588.0,01,07,0.933333,0.806452,1438.0,1417.0
57,2018,R4X1,1242.0,1222.0,1242.0,Kansas,Houston,3603.0,3717.0,04,03,0.741935,0.935484,1242.0,1257.0
58,2018,R4Y1,1181.0,1277.0,1181.0,Duke,Michigan St,4183.0,3940.0,01,02,0.838710,0.806452,1277.0,1242.0
59,2018,R4Z1,1211.0,1285.0,1211.0,Gonzaga,Montana,3996.0,3214.0,01,15,0.909091,0.724138,1211.0,1211.0


In [384]:
teamseed5 = {}
for row in range(56,60):
    a = Round4['Slot'].loc[row]
    b = Round4['NextForest'].loc[row]
    teamseed5[a] = b
    
Round5 = Round5.replace(teamseed5)

for each in range(60,62):
    a = team_scores_2019[team_scores_2019['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['Score']
    b = team_scores_2019[team_scores_2019['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['Score']
    c = aseeds[aseeds['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['Seed']
    d = aseeds[aseeds['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['Seed']
    e = record2019[record2019['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['% Win']
    f = record2019[record2019['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['% Win']
    g = teams2[teams2['TeamID'] == Round5['StrongSeed'].loc[each]].iloc[0]['TeamName']
    h = teams2[teams2['TeamID'] == Round5['WeakSeed'].loc[each]].iloc[0]['TeamName']


    Round5['StrongScore'].loc[each] = a
    Round5['WeakScore'].loc[each] = b
    Round5['HighSeed'].loc[each] = c
    Round5['LowSeed'].loc[each] = d
    Round5['High %'].loc[each] = e
    Round5['Low %'].loc[each] = f
    Round5['StrongName'].loc[each] = g
    Round5['Weakname'].loc[each] = h

U = Round5[ncaa_features]

Round5['NextForest'] = Round5['WeakSeed']
Round5['OutcomeForest'] = ncaa_model.predict(U)

for row in range(60,62):
    a = Round5['StrongSeed'].loc[row]
    b = Round5['WeakSeed'].loc[row]
    c = Round5['OutcomeForest'].loc[row]
    d = abs(a - c)
    e = abs(b - c)
    
    if d > e:
        Round5['NextForest'].loc[row] = b
    else:
        Round5['NextForest'].loc[row] = a

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [385]:
Round5

,Season,Slot,StrongSeed,WeakSeed,WTeamID,StrongName,Weakname,StrongScore,WeakScore,HighSeed,LowSeed,High %,Low %,NextForest,OutcomeForest
60,2018,R5WX,1438.0,1242.0,1438.0,Virginia,Kansas,3162.0,3603.0,01,04,0.933333,0.741935,1438.0,1438.0
61,2018,R5YZ,1277.0,1211.0,1277.0,Michigan St,Gonzaga,3940.0,3996.0,02,01,0.806452,0.909091,1211.0,1139.0


Season         2018
Slot           R6CH
StrongSeed     R5WX
WeakSeed       R5YZ
WTeamID        R5WX
StrongName     R5WX
Weakname       R5WX
StrongScore    R5WX
WeakScore      R5YZ
HighSeed       R5YZ
LowSeed        R5YZ
High %         R5YZ
Low %          R5YZ
Name: 62, dtype: object

In [389]:
final = slots.loc[62]
teamseed6 = {}
for row in range(60,62):
    a = Round5['Slot'].loc[row]
    b = Round5['NextForest'].loc[row]
    teamseed6[a] = b
    
final = final.replace(teamseed6)

final['StrongScore'] = 3162
final['WeakScore'] = 3996
final['HighSeed'] = '01'
final['LowSeed'] = '01'
final['High %'] = 0.933333
final['Low %'] = 0.909091
final['StrongName'] = 'Virginia'
final['Weakname'] = 'Gonzaga'

In [390]:
final

Season             2018
Slot               R6CH
StrongSeed         1438
WeakSeed           1211
WTeamID            1438
StrongName     Virginia
Weakname        Gonzaga
StrongScore        3162
WeakScore          3996
HighSeed             01
LowSeed              01
High %         0.933333
Low %          0.909091
Name: 62, dtype: object

In [392]:
U = final[ncaa_features]
U = pd.DataFrame(U)
U = U.transpose()
final_prediction = ncaa_model.predict(U)

In [393]:
final_prediction

array([1211.])